In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../Data/www/styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [2]:
import requests 
import requests.auth
import json
#from PIL import Image
#from io import BytesIO
from IPython.display import display, Image


## Using Reddit's API

Reddit is an entertainment, social news networking service, and news website. Reddit's registered community members can submit content, such as text posts or direct links. Registered users can then vote submissions up or down to organize the posts and determine their position on the site's pages. The submissions with the most positive votes appear on the main page or the top of a category. 

Content entries are organized by areas of interest called "subreddits". The subreddit topics include news, gaming, movies, music, books, fitness, food, and photosharing, among many others.  

Of course, what really get users of Reddit to come back are the [comments](https://www.reddit.com/r/aww/comments/4v00p4/owls_are_just_cats_with_wings_think_about_it/). 

+++

But enough of this. 

Reddit's API creates a new challenge for us.  In order to use it, we need to have an account and we need to **authenticate** our requests to the API.

We are going to be doing a couple of different things here in order to authenticate our interactions with the API.  First, we will create a session. This is done using cookies. Fortunately, we do not need to worry about any of those details because `requests` will take care of it for us.

Next, we will login to the reddit API and obtain a `modhash`. A `modhash` is a token that the reddit API requires to help prevent `Cross-Site Request Forgery` (CSRF), that is, an attack that forces an end user to execute unwanted actions on a web application in which they're currently authenticated. 

The following code was taken from https://github.com/reddit/reddit/wiki/OAuth2-Quick-Start-Example

In [5]:
# app info
app_name = 'nico101'
redirect_uri = 'http://www.example.com/unused/redirect/uri'
app_developers = ['lamaral1968']

# load your authentication information
with open('reddit_auth.json', 'r') as file_in:
    auth = json.load(file_in)
    
client_auth = requests.auth.HTTPBasicAuth(auth['app_client_ID'], auth['app_client_secret']) 
post_data = {"grant_type": "password", "username": auth['username'], "password": auth['password']}
headers = {"User-Agent": "NICO_bot/0.1 by lamaral1968"}
response = requests.post("https://www.reddit.com/api/v1/access_token", 
                         auth = client_auth, data = post_data, headers = headers)
response.json()

FileNotFoundError: [Errno 2] No such file or directory: 'reddit_auth.json'

In [ ]:
access_token = response.json()['access_token']
headers.update( {'Authorization': 'bearer ' + access_token } )
response = requests.get("https://oauth.reddit.com/api/v1/me", headers = headers)
response.json()

If all we want to do is to search Reddit for content, then we do not need to authenticate.  Let's say that we want to look for posts to Reddit that feature puppies.  How would we go about doing it?

In [ ]:
reddit_query = 'http://www.reddit.com/r/puppies/search.json'
options = {'q': 'adorable', 'sort': 'new', 'restrict_sr': 'on'}

response = requests.get( reddit_query, params = options )
print(response.url)
response.json()

In [ ]:
goodies = response.json()
print(type(goodies['data']['children']))
stories = goodies['data']['children']
print(stories[1])

`['data']['children']` is where the posts are.  We can see that we have a bunch of `keys` storing different types of information for each post.  We can look into more detail into the contents of our stories/posts.

In [ ]:
print(stories[1].keys())

In [ ]:
print(stories[0]['kind'])
print(stories[0]['data'].keys())


In [ ]:
print(stories[0]['data']['author'])
print(stories[0]['data']['id'])
print(stories[0]['data']['score'])
print(stories[0]['data']['thumbnail'])

An interesting bit of information is that posts with images have URLs stored under the key `thumbnail`. Let's select all posts that have pictures.


In [ ]:
for i, story in enumerate(stories):
    if 'http' in story['data']['thumbnail']:
        figure_url = story['data']['thumbnail']
        print(i, story['data']['name'], story['data']['created_utc'])
        display(Image(url=figure_url))

In [ ]:
index = 4
print(stories[index]['kind'])
print(stories[index]['data']['id'])
print(stories[index]['data']['score'])
print(stories[index]['data']['name'])
print('http://www.reddit.com/' + stories[index]['data']['id'])

That one is clearly too cute to leave us unmoved. **We must upvote that post**, but how can we do it?

First, it turns out you cannot vote on posts that are more than 6 months old.  All posts are archived after 6 months. So, we need to find a post that is younger than 6 months.  

Fortunately, we've already checked the date and that post is from April 23, 2016.



In [ ]:
print(stories[index]['data']['name'])

Next, we need to obtain all relevant information for voting. From the developer pages we learn that: 

    POST /api/vote  Cast a vote on a thing.

    id should be the fullname of the Link or Comment to vote on.

    dir indicates the direction of the vote. Voting 1 is an upvote, -1 is a downvote, and 0 is equivalent to "un-voting" by clicking again on a highlighted arrow.have to obtain the URL of the post.

`fullname` is created by adding the `kind` and the `id`.

In [ ]:
name = stories[index]['data']['name']
vote = requests.post('https://oauth.reddit.com/api/vote', headers = headers, params = {'id': name, 'dir': 1} )
print(vote.url)
print(vote.status_code)
print(vote.text)

Excellent! We've now voted on that adorable post! We've also successfully `POST`ed data using an API for the first time also. 

While we've had fun exploring this example, all APIs work on these exact same principles (some are just documented better than others) so you're now equipped to work with APIs all over the web!